In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/Bitewise DB - Master product list _ Ingredients & Nutrition facts - Master list.csv')

In [8]:
ingredient_list = df['ingredient'].values.tolist()

In [9]:
ingredient_list

['gạo (99%), muối tinh luyện (0.6%), rượu etylic (Ethy Alcohol (0.3%)), chất điều chỉnh độ (Acid Citric (0.1%)))',
 'Gạo (87.8%), nước, cồn thực phẩm, muối',
 'Mồng tơi 66% mướp 34%',
 'Cà rốt 38%, bắp hạt 25%, đậu que 25%, đậu hà lan 12%',
 'Khoai mì 100%',
 'Sả bào 75%, nghệ 13%, củ tỏi 6%, ớt trái 6%',
 'Khoai mì 100%',
 'Củ hủ dùa 100%',
 'Bí đỏ 96%, ngò ôm 25%, ngò gai 2%',
 'Tiêu xanh 50%, củ tỏi 17%, củ hành tím 17%, ớt trái 16%',
 'ĐẬU TRẮNG, ĐẬU ĐỎ, ĐẬU NGỰ VÀ ĐẬU HÀ LAN',
 'Đường, Chất điều chỉnh độ acid (INS 330), Hương đào tổng hợp, Tinh chất trà đen (0,7 %), Màu tổng hợp (Caramen nhóm IV - INS 150d), Muối i-ốt, Vitamin C (Acid L-ascorbic) (0,3 %) và Chất tạo ngọt tổng hợp (Aspartam-INS 951, Acesulfam kali - INS 950, Sucralose - INS 955). Sản phẩm có chứa sulphite',
 'nước',
 'NƯỚC, ĐẠI MẠCH, HOA BIA',
 'NƯỚC, ĐẠI MẠCH, NGŨ CỐC, HOA BIA',
 'Nước, mạch nha, lúa mì, đường, nước ép táo cô đặc, nước ép quả mâm xôi cô đặc, nước ép cà rốt cô đặc, hương thực phẩm tự nhiên, hoa Hou

In [21]:
import re

def masking_percent(text, subst="s7ds6d_percentage_token"):
    regex = r"\(?\d+(?:[,\.]\d+)? ?%\)?"

    # You can manually specify the number of replacements by changing the 4th argument
    result = re.sub(regex, subst, text, 0, re.MULTILINE | re.IGNORECASE | re.UNICODE)
    return result


In [22]:
mask_perc_list = [masking_percent(ingredient, '') for ingredient in ingredient_list]

In [104]:
def masking_density(text, subst="s7ds6d_density_token"):
    regex = r"\d+(?:[,\.]\d+)? ?(?:mcg|mg|kg|g) ?\/ ?(?:ml|lít|l|kg)"
    result = re.sub(regex, subst, text, 0, re.MULTILINE | re.IGNORECASE | re.UNICODE)

    regex = r"\d+(?:[,\.]\d+)? ?(?:mcg|mg|kg|g)"
    result = re.sub(regex, subst, result, 0, re.MULTILINE | re.IGNORECASE | re.UNICODE)

    regex = r"\be (\d+)"
    subst = "e\\1"
    result = re.sub(regex, subst, result, 0, re.MULTILINE | re.IGNORECASE | re.UNICODE)

    # additive variant e.g e451 (i)
    regex = r"(e?\d+) ?\(?([ivx]+)\)?"
    subst = "\\1\\2"
    result = re.sub(regex, subst, result, 0, re.MULTILINE | re.IGNORECASE | re.UNICODE)
    return result

In [105]:
clean_density_list = [masking_density(ingredient, '') for ingredient in mask_perc_list]

In [106]:
# clean_density_list

In [107]:
def extract_only_word(text):
    return " ".join(re.findall(r"[\w|\d']+", text))

In [108]:
def split_into_term(text):
    pattern = r"\s*(?:,|;|\(|\)|\.|và|\:| \- |\*|\[|\]|\&|\!|\@|\#|\^|\/|\\|\|)\s*"
    terms = re.split(pattern, text, flags=re.MULTILINE | re.IGNORECASE | re.UNICODE)
    return [extract_only_word(i.strip().lower()) for i in terms if i.strip() != '']

In [109]:
split_into_term_list = [split_into_term(ingredient) for ingredient in clean_density_list]

In [110]:
split_into_term_list

[['gạo',
  'muối tinh luyện',
  'rượu etylic',
  'ethy alcohol',
  'chất điều chỉnh độ',
  'acid citric'],
 ['gạo', 'nước', 'cồn thực phẩm', 'muối'],
 ['mồng tơi mướp'],
 ['cà rốt', 'bắp hạt', 'đậu que', 'đậu hà lan'],
 ['khoai mì'],
 ['sả bào', 'nghệ', 'củ tỏi', 'ớt trái'],
 ['khoai mì'],
 ['củ hủ dùa'],
 ['bí đỏ', 'ngò ôm', 'ngò gai'],
 ['tiêu xanh', 'củ tỏi', 'củ hành tím', 'ớt trái'],
 ['đậu trắng', 'đậu đỏ', 'đậu ngự', 'đậu hà lan'],
 ['đường',
  'chất điều chỉnh độ acid',
  'ins 330',
  'hương đào tổng hợp',
  'tinh chất trà đen',
  'màu tổng hợp',
  'caramen nhóm iv',
  'ins 150d',
  'muối i ốt',
  'vitamin c',
  'acid l ascorbic',
  'chất tạo ngọt tổng hợp',
  'aspartam ins 951',
  'acesulfam kali',
  'ins 950',
  'sucralose',
  'ins 955',
  'sản phẩm có chứa sulphite'],
 ['nước'],
 ['nước', 'đại mạch', 'hoa bia'],
 ['nước', 'đại mạch', 'ngũ cốc', 'hoa bia'],
 ['nước',
  'mạch nha',
  'lúa mì',
  'đường',
  'nước ép táo cô đặc',
  'nước ép quả mâm xôi cô đặc',
  'nước ép cà rốt

In [111]:
tokenize_sent_list = list(set([" | ".join(ingredient) for ingredient in split_into_term_list]))


In [112]:
# write to file
with open('data/ingredient_token.txt', 'w') as f:
    for item in tokenize_sent_list:
        f.write("%s\n" % item)

In [113]:
merge_terms = set([term for ingredient in split_into_term_list for term in ingredient])

In [114]:
# write to file
with open('data/merge_terms.txt', 'w') as f:
    for item in merge_terms:
        f.write("%s\n" % item)